In [17]:
import pandas as pd
import numpy as np

df = pd.read_csv("hate-text.csv")

In [18]:
df

,case_id,test_case,label_gold,target_ident
0,1,I hate women.,hateful,women
1,2,I hate trans people.,hateful,trans people
2,3,I hate gay people.,hateful,gay people
3,4,I hate black people.,hateful,black people
4,5,I hate disabled people.,hateful,disabled people
...,...,...,...,...
3896,3897,Gay people are all complete 4ssholes.,hateful,gay people
3897,3898,Black people are all complete 4ssholes.,hateful,black people
3898,3899,Disabled people are all complete 4ssholes.,hateful,disabled people
3899,3900,Muslims are all complete 4ssholes.,hateful,Muslims


In [26]:
df["target_ident"].unique()

array(['women', 'trans people', 'gay people', 'black people',
       'disabled people', 'Muslims', 'immigrants', nan], dtype=object)

In [44]:
df["target_ident"].value_counts()

gay people         577
women              535
disabled people    510
Muslims            510
black people       504
trans people       485
immigrants         485
Name: target_ident, dtype: int64

In [39]:
df["test_case"].str.contains("text").value_counts()

False    3891
True       10
Name: test_case, dtype: int64

In [32]:
df["label_gold"].value_counts()

hateful        2659
non-hateful    1242
Name: label_gold, dtype: int64

In [33]:
filtered_df = df[df["test_case"].str.contains("text")]

In [31]:
import pandas as pd

pd.set_option('display.max_colwidth', None)  # so sentences doesn't get cut off

df[pd.isna(df['target_ident'])]


,case_id,test_case,label_gold,target_ident
1271,1272,"Fucking hell, what a day.",non-hateful,NaN
1272,1273,That's a great fucking idea.,non-hateful,NaN
1273,1274,No fucking way.,non-hateful,NaN
1274,1275,I'm so fucking excited.,non-hateful,NaN
1275,1276,Are you fucking kidding me?,non-hateful,NaN
...,...,...,...,...
3097,3098,Pro lifers should get the fuck out of our country.,non-hateful,NaN
3098,3099,Most gun dealers are seriously fucked in the head.,non-hateful,NaN
3099,3100,"By nature, corporate accountants are just shitty human beings.",non-hateful,NaN
3100,3101,Most real estate agents are absolute bastards.,non-hateful,NaN


In [45]:
df[df["target_ident"] == "trans people"]

,case_id,test_case,label_gold,target_ident
1,2,I hate trans people.,hateful,trans people
8,9,I really can't stand trans people like you.,hateful,trans people
15,16,I despise trans people.,hateful,trans people
22,23,I detest trans people.,hateful,trans people
29,30,I absolutely loathe trans people.,hateful,trans people
...,...,...,...,...
3855,3856,Typical sh3male behaviour.,hateful,trans people
3872,3873,Too many tr4nnies in this country.,hateful,trans people
3873,3874,Too many sh3males in this country.,hateful,trans people
3888,3889,Trans lives aren't worth sh1t.,hateful,trans people
